# Run learners in job scripts

## Define the learners

We need the following variables:
* `learners` a list of learners
* `combos` a list of dicts of parameters that describe each learner
* `fnames` a list of filenames of each learner

In [ ]:
%%writefile learners_file.py

import adaptive
from functools import partial


def h(x, offset=0):
    import numpy as np
    import random

    for _ in range(10):  # Burn some CPU time just because
        np.linalg.eig(np.random.rand(1000, 1000))

    a = 0.01
    return x + a ** 2 / (a ** 2 + (x - offset) ** 2)


offset = [i / 20 - 0.5 for i in range(20)]

combos = adaptive.utils.named_product(offset=offset)

learners = []
fnames = []

folder = "data/"

for i, combo in enumerate(combos):
    f = partial(h, offset=combo["offset"])
    learner = adaptive.Learner1D(f, bounds=(-1, 1))
    fnames.append(f"{folder}{combo}")
    learners.append(learner)

learner = adaptive.BalancingLearner(learners)

In [ ]:
# Execute the previous code block and plot the learners
from learners_file import *
adaptive.notebook_extension()
learner.load(fnames)
learner.plot()

## The Python script that is being run in the job

In [ ]:
# Make sure to use the headnode's address in the next cell
from adaptive_scheduler import server_support
server_support.get_allowed_url()

In [ ]:
%%writefile run_learner.py

import adaptive
from adaptive_scheduler import client_support
from mpi4py.futures import MPIPoolExecutor

from learners_file import learners, combos

url = "tcp://10.75.0.5:37371"
learner, fname = client_support.get_learner(url, learners, combos)
learner.load(fname)
runner = adaptive.Runner(
    learner, executor=MPIPoolExecutor(), shutdown_executor=True, goal=None
)
runner.start_periodic_saving(dict(fname=fname), interval=600)
runner.ioloop.run_until_complete(runner.task)  # wait until runner goal reached
client_support.is_done(url, fname)

# Import the files that were created

In [ ]:
from adaptive_scheduler import server_support
from tinydb import TinyDB

import learners_file, run_learner

# Reload the files in case you overwrote them
from importlib import reload
reload(learners_file)
reload(run_learner)

db_fname = 'running.tinydb'

In [ ]:
# Create a new database
server_support.create_empty_db(db_fname, learners_file.fnames, learners_file.combos)

## Check the running learners
All the onces that are `None` are still `PENDING`, reached their goal, or are not scheduled.

In [ ]:
server_support.get_database(db_fname)

## Start the job scripts

In [ ]:
import asyncio
from learners_file import learners, combos, fname

# create unique names for the jobs
job_names = [f"test-job-{i}" for i in range(len(learners))]

# start the "job manager" and the "database manager"
ioloop = asyncio.get_event_loop()

database_task = ioloop.create_task(
    server_support.manage_database("tcp://10.75.0.5:37371", db_fname)
)

job_task = ioloop.create_task(
    server_support.manage_jobs(
        job_names,
        db_fname=db_fname,
        ioloop=ioloop,
        cores=16,
        interval=60,
        run_script="run_learner.py",  # optional
        python_executable="~/miniconda3/envs/py37_min/bin/python", # optional
        job_script_function=adaptive_scheduler.slurm.make_sbatch,  # optional
    )
)

In [ ]:
job_task.print_stack()

In [ ]:
database_task.print_stack()

In [ ]:
# Run this to STOP managing the database and jobs
job_task.cancel(), database_task.cancel()